In [1]:
# !conda activate n2v
%load_ext autoreload
from noise2gun import *
from compare import *

2021-08-23 10:18:29.286131: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


# Define and Load datasets
Must be ZARRs

## Define

In [4]:
src_path = '.../volumes'
gt_name = 'raw'
ds_names = [NOT CYCLEGAN]


## Load

In [ ]:
comparator = Compare(src_path, # 'path/to/data.zarr/volumes'
                gt_name=gt_name, # 'gt_dataset_name'
                ds_names=ds_names, # ['dataset_1_name', 'dataset_2_name', ...]
                out_path=None,
                batch_size=1,
                metric_list=None,
                vizualize=True
    )